In [7]:
import pandas as pd
import sqlalchemy
import numpy as np
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey
from sqlalchemy.ext.declarative import declarative_base

### Extract CSVs into DataFrames

In [8]:
# c = confirmed     d = death     r = recovered
cww_file = "confirmed_global.csv"
dww_file = "deaths_global.csv"
rww_file = "recovered_global.csv"

### Transform DataFrames

In [15]:
cww_data = pd.read_csv(cww_file)
cww_df = pd.DataFrame(cww_data)
cww_df = cww_df.dropna(how='all') 
cww_df.insert(0, "fips", "NA")
cww_df= cww_df.rename(columns={"Province/State": "estado", "Country/Region": "pais"})
#The next line is to add consecutive numbers
cww_df['row_number'] = np.arange(len(cww_df))
cww_df.head(10)

,fips,estado,pais,status,Lat,Long,January_22,January_23,January_24,January_25,...,April_14,April_15,April_16,April_17,April_18,April_19,April_20,April_21,April_22,row_number
0,NA,NaN,Afghanistan,confirmed,33.0000,65.0000,0,0,0,0,...,714,784,840,906,933,996,1026,1092,1176,0
1,NA,NaN,Albania,confirmed,41.1533,20.1683,0,0,0,0,...,475,494,518,539,548,562,584,609,634,1
2,NA,NaN,Algeria,confirmed,28.0339,1.6596,0,0,0,0,...,2070,2160,2268,2418,2534,2629,2718,2811,2910,2
3,NA,NaN,Andorra,confirmed,42.5063,1.5218,0,0,0,0,...,659,673,673,696,704,713,717,717,723,3
4,NA,NaN,Angola,confirmed,-11.2027,17.8739,0,0,0,0,...,19,19,19,19,24,24,24,24,25,4
5,NA,NaN,Antigua and Barbuda,confirmed,17.0608,-61.7964,0,0,0,0,...,23,23,23,23,23,23,23,23,24,5
6,NA,NaN,Argentina,confirmed,-38.4161,-63.6167,0,0,0,0,...,2277,2443,2571,2669,2758,2839,2941,3031,3144,6
7,NA,NaN,Armenia,confirmed,40.0691,45.0382,0,0,0,0,...,1067,1111,1159,1201,1248,1291,1339,1401,1473,7
8,NA,Australian Capital Territory,Australia,confirmed,-35.4735,149.0124,0,0,0,0,...,103,103,103,103,103,103,104,104,104,8
9,NA,New South Wales,Australia,confirmed,-33.8688,151.2093,0,0,0,0,...,2870,2886,2897,2926,2936,2957,2963,2969,2971,9


In [14]:
dww_data = pd.read_csv(dww_file)
dww_df = pd.DataFrame(dww_data)
dww_df = dww_df.dropna(how='all') 
dww_df.insert(0, "fips", "NA")
dww_df= dww_df.rename(columns={"Province/State": "estado", "Country/Region": "pais"})
#The next line is to add consecutive numbers
dww_df['row_number'] = np.arange(len(dww_df))
dww_df.head(10)

,fips,estado,pais,status,Lat,Long,January_22,January_23,January_24,January_25,...,April_14,April_15,April_16,April_17,April_18,April_19,April_20,April_21,April_22,row_number
0,NA,NaN,Afghanistan,death,33.0000,65.0000,0,0,0,0,...,23,25,30,30,30,33,36,36,40,0
1,NA,NaN,Albania,death,41.1533,20.1683,0,0,0,0,...,24,25,26,26,26,26,26,26,27,1
2,NA,NaN,Algeria,death,28.0339,1.6596,0,0,0,0,...,326,336,348,364,367,375,384,392,402,2
3,NA,NaN,Andorra,death,42.5063,1.5218,0,0,0,0,...,31,33,33,35,35,36,37,37,37,3
4,NA,NaN,Angola,death,-11.2027,17.8739,0,0,0,0,...,2,2,2,2,2,2,2,2,2,4
5,NA,NaN,Antigua and Barbuda,death,17.0608,-61.7964,0,0,0,0,...,2,2,3,3,3,3,3,3,3,5
6,NA,NaN,Argentina,death,-38.4161,-63.6167,0,0,0,0,...,102,111,115,123,129,132,136,147,152,6
7,NA,NaN,Armenia,death,40.0691,45.0382,0,0,0,0,...,16,17,18,19,20,20,22,24,24,7
8,NA,Australian Capital Territory,Australia,death,-35.4735,149.0124,0,0,0,0,...,2,3,3,3,3,3,3,3,3,8
9,NA,New South Wales,Australia,death,-33.8688,151.2093,0,0,0,0,...,25,25,25,26,26,26,26,26,26,9


In [13]:
rww_data = pd.read_csv(rww_file)
rww_df = pd.DataFrame(rww_data)
rww_df = rww_df.dropna(how='all') 
rww_df.insert(0, "fips", "NA")
rww_df= rww_df.rename(columns={"Province/State": "estado", "Country/Region": "pais"})
#The next line is to add consecutive numbers
rww_df['row_number'] = np.arange(len(rww_df))
rww_df.head(10)

,fips,estado,pais,status,Lat,Long,January_22,January_23,January_24,January_25,...,April_14,April_15,April_16,April_17,April_18,April_19,April_20,April_21,April_22,row_number
0,NA,NaN,Afghanistan,recovered,33.0000,65.0000,0,0,0,0,...,40,43,54,99,112,131,135,150,166,0
1,NA,NaN,Albania,recovered,41.1533,20.1683,0,0,0,0,...,248,251,277,283,302,314,327,345,356,1
2,NA,NaN,Algeria,recovered,28.0339,1.6596,0,0,0,0,...,691,708,783,846,894,1047,1099,1152,1204,2
3,NA,NaN,Andorra,recovered,42.5063,1.5218,0,0,0,0,...,128,169,169,191,205,235,248,282,309,3
4,NA,NaN,Angola,recovered,-11.2027,17.8739,0,0,0,0,...,5,5,5,5,6,6,6,6,6,4
5,NA,NaN,Antigua and Barbuda,recovered,17.0608,-61.7964,0,0,0,0,...,3,3,3,3,3,3,3,7,10,5
6,NA,NaN,Argentina,recovered,-38.4161,-63.6167,0,0,0,0,...,559,596,631,666,685,709,737,840,872,6
7,NA,NaN,Armenia,recovered,40.0691,45.0382,0,0,0,0,...,265,297,358,402,523,545,580,609,633,7
8,NA,Australian Capital Territory,Australia,recovered,-35.4735,149.0124,0,0,0,0,...,72,72,82,81,88,88,88,88,88,8
9,NA,New South Wales,Australia,recovered,-33.8688,151.2093,0,0,0,0,...,4,4,4,1379,1379,1379,1379,1379,1379,9


### Merge all DataFrames

In [16]:
#Merge by state
ww_df = pd.concat([cww_df, dww_df, rww_df], ignore_index=True)
#The next line is to add consecutive numbers
ww_df['row_number'] = np.arange(len(ww_df))
ww_df.head()

,fips,estado,pais,status,Lat,Long,January_22,January_23,January_24,January_25,...,April_14,April_15,April_16,April_17,April_18,April_19,April_20,April_21,April_22,row_number
0,NA,NaN,Afghanistan,confirmed,33.0000,65.0000,0,0,0,0,...,714,784,840,906,933,996,1026,1092,1176,0
1,NA,NaN,Albania,confirmed,41.1533,20.1683,0,0,0,0,...,475,494,518,539,548,562,584,609,634,1
2,NA,NaN,Algeria,confirmed,28.0339,1.6596,0,0,0,0,...,2070,2160,2268,2418,2534,2629,2718,2811,2910,2
3,NA,NaN,Andorra,confirmed,42.5063,1.5218,0,0,0,0,...,659,673,673,696,704,713,717,717,723,3
4,NA,NaN,Angola,confirmed,-11.2027,17.8739,0,0,0,0,...,19,19,19,19,24,24,24,24,25,4


### Create database connection

In [36]:
rds_connection_string = "postgres:postgres@localhost:5432/Project2"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [37]:
# Confirm tables
engine.table_names()

['negativos', 'defunciones', 'confirmados', 'estados', 'nacional']

### Load DataFrames into database

In [38]:
cww_df.to_sql(name='confirmed', con=engine, if_exists='append', index=False)

In [39]:
dww_df.to_sql(name='death', con=engine, if_exists='append', index=False)

In [40]:
rww_df.to_sql(name='recovered', con=engine, if_exists='append', index=False)

In [41]:
ww_df.to_sql(name='all_countries', con=engine, if_exists='append', index=False)

In [42]:
#Print the sql table with a query
pd.read_sql_query('select * from confirmed', con=engine).head()

,fips,estado,pais,status,Lat,Long,January_22,January_23,January_24,January_25,...,April_14,April_15,April_16,April_17,April_18,April_19,April_20,April_21,April_22,row_number
0,NA,None,Afghanistan,confirmed,33.0000,65.0000,0,0,0,0,...,714,784,840,906,933,996,1026,1092,1176,0
1,NA,None,Albania,confirmed,41.1533,20.1683,0,0,0,0,...,475,494,518,539,548,562,584,609,634,1
2,NA,None,Algeria,confirmed,28.0339,1.6596,0,0,0,0,...,2070,2160,2268,2418,2534,2629,2718,2811,2910,2
3,NA,None,Andorra,confirmed,42.5063,1.5218,0,0,0,0,...,659,673,673,696,704,713,717,717,723,3
4,NA,None,Angola,confirmed,-11.2027,17.8739,0,0,0,0,...,19,19,19,19,24,24,24,24,25,4


In [43]:
pd.read_sql_query('select * from recovered', con=engine).head()

,fips,estado,pais,status,Lat,Long,January_22,January_23,January_24,January_25,...,April_14,April_15,April_16,April_17,April_18,April_19,April_20,April_21,April_22,row_number
0,NA,None,Afghanistan,recovered,33.0000,65.0000,0,0,0,0,...,40,43,54,99,112,131,135,150,166,0
1,NA,None,Albania,recovered,41.1533,20.1683,0,0,0,0,...,248,251,277,283,302,314,327,345,356,1
2,NA,None,Algeria,recovered,28.0339,1.6596,0,0,0,0,...,691,708,783,846,894,1047,1099,1152,1204,2
3,NA,None,Andorra,recovered,42.5063,1.5218,0,0,0,0,...,128,169,169,191,205,235,248,282,309,3
4,NA,None,Angola,recovered,-11.2027,17.8739,0,0,0,0,...,5,5,5,5,6,6,6,6,6,4


In [44]:
pd.read_sql_query('select * from death', con=engine).head()

,fips,estado,pais,status,Lat,Long,January_22,January_23,January_24,January_25,...,April_14,April_15,April_16,April_17,April_18,April_19,April_20,April_21,April_22,row_number
0,NA,None,Afghanistan,death,33.0000,65.0000,0,0,0,0,...,23,25,30,30,30,33,36,36,40,0
1,NA,None,Albania,death,41.1533,20.1683,0,0,0,0,...,24,25,26,26,26,26,26,26,27,1
2,NA,None,Algeria,death,28.0339,1.6596,0,0,0,0,...,326,336,348,364,367,375,384,392,402,2
3,NA,None,Andorra,death,42.5063,1.5218,0,0,0,0,...,31,33,33,35,35,36,37,37,37,3
4,NA,None,Angola,death,-11.2027,17.8739,0,0,0,0,...,2,2,2,2,2,2,2,2,2,4


In [45]:
pd.read_sql_query('select * from all_countries', con=engine).head()

,fips,estado,pais,status,Lat,Long,January_22,January_23,January_24,January_25,...,April_14,April_15,April_16,April_17,April_18,April_19,April_20,April_21,April_22,row_number
0,NA,None,Afghanistan,confirmed,33.0000,65.0000,0,0,0,0,...,714,784,840,906,933,996,1026,1092,1176,0
1,NA,None,Albania,confirmed,41.1533,20.1683,0,0,0,0,...,475,494,518,539,548,562,584,609,634,1
2,NA,None,Algeria,confirmed,28.0339,1.6596,0,0,0,0,...,2070,2160,2268,2418,2534,2629,2718,2811,2910,2
3,NA,None,Andorra,confirmed,42.5063,1.5218,0,0,0,0,...,659,673,673,696,704,713,717,717,723,3
4,NA,None,Angola,confirmed,-11.2027,17.8739,0,0,0,0,...,19,19,19,19,24,24,24,24,25,4


### SQL Tables

In [46]:
#Set a primary key in an existing table
engine.execute('alter table confirmed add primary key(row_number)')

In [47]:
engine.execute('alter table recovered add primary key(row_number)')

In [48]:
engine.execute('alter table death add primary key(row_number)')

In [49]:
engine.execute('alter table all_countries add primary key(row_number)')

In [50]:
#THis is just to check if the tables are been mapped
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [51]:
Base.classes.keys()

['recovered',
 'confirmed',
 'death',
 'all_countries',
 'negativos',
 'defunciones',
 'confirmados',
 'estados',
 'nacional']

# Just change the table name 
create table confirmed, death, recovered, all
(
fips varchar(100)
estado varchar(100),
pais varchar(100),
status varchar(100),
Lat int,
Long int,
fips varchar(100)
estado varchar(100),
pais varchar(100),
status varchar(100),
Lat int,
Long int,
"January_22" string,
"January_23" string,
"January_24" string,
"January_25" string,
"January_26" string,
"January_27" string,
"January_28" string,
"January_29" string,
"January_30" string,
"January_31" string,
"Feburary_1" string,
"Feburary_2" string,
"Feburary_3" string,
"Feburary_4" string,
"Feburary_5" string,
"Feburary_6" string,
"Feburary_7" string,
"Feburary_8" string,
"Feburary_9" string,
"Feburary_10" string,
"Feburary_11" string,
"Feburary_12" string,
"Feburary_13" string,
"Feburary_14" string,
"Feburary_15" string,
"Feburary_16" string,
"Feburary_17" string,
"Feburary_18" string,
"Feburary_19" string,
"Feburary_20" string,
"Feburary_21" string,
"Feburary_22" string,
"Feburary_23" string,
"Feburary_24" string,
"Feburary_25" string,
"Feburary_26" string,
"Feburary_27" string,
"Feburary_28" string,
"Feburary_29" string,
"March_1" string,
"March_2" string,
"March_3" string,
"March_4" string,
"March_5" string,
"March_6" string,
"March_7" string,
"March_8" string,
"March_9" string,
"March_10" string,
"March_11" string,
"March_12" string,
"March_13" string,
"March_14" string,
"March_15" string,
"March_16" string,
"March_17" string,
"March_18" string,
"March_19" string,
"March_20" string,
"March_21" string,
"March_22" string,
"March_23" string,
"March_24" string,
"March_25" string,
"March_26" string,
"March_27" string,
"March_28" string,
"March_29" string,
"March_30" string,
"March_31" string,
"April_1" string,
"April_2" string,
"April_3" string,
"April_4" string,
"April_5" string,
"April_6" string,
"April_7" string,
"April_8" string,
"April_9" string,
"April_10" string,
"April_11" string,
"April_12" string,
"April_13" string,
"April_14" string,
"April_15" string,
"April_16" string,
"April_17" string,
"April_18" string,
"April_19" string,
"April_20" string,
"April_21" string,
"April_22" string
)